In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import os
from sklearn.metrics import confusion_matrix

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer, TfidfVectorizer
import pandas as pd
import numpy as np

import tensorflow as tf
from tensorflow.python.keras import layers, models
from sklearn.feature_extraction.text import TfidfTransformer
from tensorflow import keras
from sklearn import preprocessing
from sklearn.preprocessing import OneHotEncoder
from own_functions import reading_in, pre_pros_y, tokenize_corpus, read_synth, add_together, ready_CNN
from sklearn.utils import shuffle
import mlflow
import tensorflow_addons as tfa
from tqdm import tqdm
import random
from sklearn.naive_bayes import GaussianNB


# Run CNNs

In [ ]:
cat_lab = {'emoji': 'emoji','emojii': 'emoji',

'code-switching': 'other languages','foreign language': 'other languages','different languages': 'other languages',

'typos': 'spelling and punctuation','spelling': 'spelling and punctuation','spelling error': 'spelling and punctuation',
'misspelling': 'spelling and punctuation',"{'typos'}": 'spelling and punctuation',"{'typos', 'punct'}": 'spelling and punctuation',
'typos, expand contractions, remove punctuation': 'spelling and punctuation',"{'punct'}": 'spelling and punctuation','typo': 'spelling and punctuation',

'irony': 'irony or sarcasm','irony/sarcasm': 'irony or sarcasm','sarcasm': 'irony or sarcasm','ironic': 'irony or sarcasm',

'negation': 'negation','negation (not + pos_adj)': 'negation','negation at the end': 'negation','short sentence with negation + postive adjective': 'negation',

'double negation': 'double negation','negated negatives': 'double negation',

'double meaning': 'confusing','confusing': 'confusing','hard to interpret': 'confusing','slang': 'confusing','vague': 'confusing','basic': 'confusing',

'strong words': 'exaggeration','exaggeration': 'exaggeration',

'replacement': 'vocabulary','synonym': 'vocabulary','ambigious words': 'vocabulary','rare words': 'vocabulary','infrequent word': 'vocabulary',

'sentiment change over time': 'sentiment change in text','sentiment over time': 'sentiment change in text','temporal': 'sentiment change in text','hopes vs reality': 'sentiment change in text',
'past and present thoughts': 'sentiment change in text','time change': 'sentiment change in text'

}

In [ ]:
mlflow.set_experiment("NLP project - CNN without embeddings")

tf.compat.v1.logging.set_verbosity(tf.compat.v1.logging.ERROR) #Filter info and warning messages
random.seed(42)

train = reading_in('classification/music_reviews_train.json.gz')
dev = reading_in('classification/music_reviews_dev.json.gz')
dif = reading_in("classification/all difficult cases.txt.gz")

In [ ]:
want = []
for i in dif:
    if i["category"].lower().strip() in cat_lab.keys():
        want.append(i)

In [ ]:
dif_x = [i['reviewText'] for i in want if "reviewText" in i.keys() and "sentiment" in i.keys()]
dif_y =  [i['sentiment'] for i in want if "reviewText" in i.keys() and "sentiment" in i.keys()]
dif_cat_notthisone = [i['category'].lower().strip() for i in want if "reviewText" in i.keys() and "sentiment" in i.keys()]

In [ ]:
train_x = [i['reviewText'] for i in train if "reviewText" in i.keys() and "sentiment" in i.keys()]
train_y =  [i['sentiment'] for i in train if "reviewText" in i.keys() and "sentiment" in i.keys()]

dev_x = [i['reviewText'] if "reviewText" in i.keys() else "" for i in dev]
dev_y = [i['sentiment'] for i in dev]

In [ ]:
###split into pos and neg
train_x_pos = []
train_x_neg = []
train_y_pos = []
train_y_neg = []
for i in range(len(train_x)):
    if train_y[i] == "positive":
        train_x_pos.append(train_x[i])
        train_y_pos.append(train_y[i])
    else:
        train_x_neg.append(train_x[i])
        train_y_neg.append(train_y[i])
train_x_neg.extend(train_x_pos[0:len(train_x_neg)])
train_y_neg.extend(train_y_pos[0:len(train_y_neg)])
balanced_x_train_s = train_x_neg
balanced_y_train_s = train_y_neg

In [ ]:
balanced_x_train, balanced_y_train = shuffle(balanced_x_train_s, balanced_y_train_s, random_state= 42)
dev_x, dev_y = shuffle(dev_x, dev_y, random_state = 42)
#Tokenize X data
balanced_x_train = tokenize_corpus(balanced_x_train)
dev_x = tokenize_corpus(dev_x)

In [ ]:
paths = ["webscrape.csv","wordnet.csv","word2vec.csv"]
deli = [" ","\t","\t"]

synthetic = dict()
names= ["web","net","vec"]
for x,i in enumerate(paths):
    synthetic[names[x]] = read_synth(i,deli[x])

In [ ]:
Scrape10_X, Scrape10_y = add_together(synthetic['web'], 10000,balanced_x_train, balanced_y_train)
Scrape5_X, Scrape5_y = add_together(synthetic['web'], 5000,balanced_x_train, balanced_y_train)
Scrape50_X, Scrape50_y = add_together(synthetic['web'], 500,balanced_x_train, balanced_y_train)
Scrape12_X, Scrape12_y = add_together(synthetic['web'], 1250,balanced_x_train, balanced_y_train)
Scrape2_X, Scrape2_y = add_together(synthetic['web'], 2500,balanced_x_train, balanced_y_train)
Scrape7_X, Scrape7_y = add_together(synthetic['web'], 7500,balanced_x_train, balanced_y_train)

Net10_X, Net10_y = add_together(synthetic['net'], 10000, balanced_x_train, balanced_y_train)
Net5_X, Net5_y = add_together(synthetic['net'], 5000, balanced_x_train, balanced_y_train)
Net7_X, Net7_y = add_together(synthetic['net'], 7500, balanced_x_train, balanced_y_train)
Net2_X, Net2_y = add_together(synthetic['net'], 2500, balanced_x_train, balanced_y_train)
Net12_X, Net12_y = add_together(synthetic['net'], 1250, balanced_x_train, balanced_y_train)
Net50_X, Net50_y = add_together(synthetic['net'], 500,balanced_x_train, balanced_y_train)

Vec10_X, Vec10_y =add_together(synthetic['vec'],10000,balanced_x_train, balanced_y_train)
Vec5_X, Vec5_y = add_together(synthetic['vec'], 5000,balanced_x_train, balanced_y_train)
Vec7_X, Vec7_y = add_together(synthetic['vec'], 7500,balanced_x_train, balanced_y_train)
Vec2_X, Vec2_y = add_together(synthetic['vec'], 2500,balanced_x_train, balanced_y_train)
Vec12_X, Vec12_y = add_together(synthetic['vec'], 1250,balanced_x_train, balanced_y_train)
Vec50_X, Vec50_y = add_together(synthetic['vec'], 500,balanced_x_train, balanced_y_train)

In [ ]:
# Training X, training y,  description
train_list = [
    [balanced_x_train[:500], balanced_y_train[:500] ,'original data 500'],
    [balanced_x_train[:1250], balanced_y_train[:1250] ,'original data 1250'],
    [balanced_x_train[:10000], balanced_y_train[:10000],'original data 10000'],
    [balanced_x_train[:5000], balanced_y_train[:5000], 'original data 5000'], 
    [balanced_x_train[:7500], balanced_y_train[:7500], 'original data 7500'], 
    [balanced_x_train[:2500], balanced_y_train[:2500], 'original data 2500'], 

    [Scrape10_X, Scrape10_y , 'WebScrape 10000'],
    [Scrape5_X, Scrape5_y,'WebScrape 5000' ],
    [Scrape50_X, Scrape50_y, 'WebScrape 500'],
    [Scrape12_X, Scrape12_y, 'WebScrape 1250'],
    [Scrape2_X, Scrape2_y, 'WebScrape 2500'],
    [Scrape7_X, Scrape7_y, 'WebScrape 7500' ],

    [Net10_X, Net10_y, 'WordNet 10000'],
    [Net5_X, Net5_y , 'WordNet 5000'],
    [Net7_X, Net7_y, 'WordNet 7500' ],
    [Net2_X, Net2_y, 'WordNet 2500'],
    [Net12_X, Net12_y , 'WordNet 1250'],
    [Net50_X, Net50_y, 'WordNet 500'],

    [Vec10_X, Vec10_y, 'Word2Vec 10000' ],
    [Vec5_X, Vec5_y, 'Word2Vec 5000' ],
    [Vec7_X, Vec7_y, 'Word2Vec 7500' ],
    [Vec2_X, Vec2_y, 'Word2Vec 2500' ],
    [Vec12_X, Vec12_y, 'Word2Vec 1250' ],
    [Vec50_X, Vec50_y, 'Word2Vec 500' ]
]

In [ ]:
callback = tf.keras.callbacks.EarlyStopping(monitor='loss', patience=3)

In [ ]:
for i in tqdm(train_list):
     with mlflow.start_run(run_name=f"CNN WE with {i[2]}"):
        
        tf.keras.backend.clear_session()
        tf.keras.backend.learning_phase()
        
        act = 'sigmoid'
        model = models.Sequential()
        model.add(layers.Conv1D(filters = 32, kernel_size = 10, strides = 5, padding = "same", activation='relu'))
        model.add(layers.AveragePooling1D(pool_size = 20, padding = "same", strides = 1))
        model.add(layers.Flatten())
        model.add(layers.Dense(256, activation=act))
        #model.add(layers.Dense(512, activation=act))
        #model.add(layers.Dense(1024, activation=act))
        model.add(layers.Dense(512, activation=act))
        model.add(layers.Dense(256, activation=act))
        model.add(layers.Dense(1, activation=act))
        
        
        #Get the data in the right format
        train_x, train_y, devt_x, devt_y, dif_temp_x, dif_temp_y = ready_CNN(i[0], i[1],dev_x, dev_y, dif_x, dif_y)
        model.compile(loss='binary_crossentropy', optimizer='adamax', metrics=['accuracy'])
        history = model.fit(train_x, train_y, epochs=10, batch_size = 500, callbacks=[callback], verbose=0) #change the numbers
        metrics = model.evaluate(devt_x, devt_y, verbose=0) #change the number
        mlflow.log_metric('binary_crossentropy', metrics[0])
        mlflow.log_metric('accuracy', metrics[1])
        mlflow.log_param('Actual epochs',len(history.history['loss']) )
        mlflow.log_param('NN size', '[256, 512, 256, 1, epoch = 10, betch = 500]')
        mlflow.log_param('activation', act)
        model.save(f'models/{i[2]}')
        
        #dataframes
        dev_pre = model.predict(devt_x)
        devt_y = np.reshape(devt_y, [len(devt_y),1])
        #print(dev_pre.shape)
        #dev_pre = dev_pre[:, 0, :]
        
        devt_y = np.transpose(devt_y, axes=None)
        dev_pre = np.transpose(dev_pre, axes=None)
        devt_y = devt_y.tolist()
        dev_pre = dev_pre.tolist()
        
        df = pd.DataFrame(list(zip(devt_y[0], dev_pre[0])),
               columns =['True', 'Predicted'])
    
        df.to_csv(f"pics/CNN/dev {i[2]}.csv")
        #################
        dif_pre = model.predict(dif_temp_x)
        dif_temp_y = np.reshape(dif_temp_y, [len(dif_temp_y),1])
        #dif_pre = dif_pre[:, 0, :]
        
        dif_temp_y = np.transpose(dif_temp_y, axes=None)
        dif_pre = np.transpose(dif_pre, axes=None)
        dif_temp_y = dif_temp_y.tolist()
        dif_pre = dif_pre.tolist()
        
        df = pd.DataFrame(list(zip(dif_temp_y[0], dif_pre[0], dif_cat_notthisone)),
               columns =['True', 'Predicted', 'Cat'])
    
        df.to_csv(f"pics/CNN/dif {i[2]}.csv")

# Run NB

In [ ]:
def ready_NB(data_X, data_y, dev_x, dev_y, dif_x, dif_y):
    #ADD TOKENIZER TO THE DEVELEPMENT DATA

    vectorizer = TfidfVectorizer()
    tfid = vectorizer.fit(data_X)
    train_x = tfid.transform(data_X).toarray()
    #train_x = np.reshape(train_x, (len(train_x), train_x.shape[1], 1))
    #train_x = tf.convert_to_tensor(train_x)

    train_y = pre_pros_y(data_y)
    
    devt_x = tfid.transform(dev_x)
    devt_x = devt_x.toarray()
    #devt_x = np.reshape(devt_x, (len(devt_x), devt_x.shape[1], 1))
    #devt_x = tf.convert_to_tensor(devt_x)

    devt_y = pre_pros_y(dev_y)

    dif_x = tfid.transform(dif_x)
    dif_x = dif_x.toarray()
    #dif_x = np.reshape(dif_x, (len(dif_x), dif_x.shape[1], 1))
    #dif_x = tf.convert_to_tensor(dif_x)

    dif_y = pre_pros_y(dif_y)
    return train_x, train_y, devt_x, devt_y, dif_x, dif_y

In [ ]:
random.seed(2)
MNB = Pipeline([
    ('CountVectorizer',CountVectorizer(ngram_range=(1, 2), tokenizer = tokenize_corpus, lowercase = False )),
    ('Tfidf',TfidfTransformer()),
    ('clf', MultinomialNB(alpha=1, fit_prior=False))
])#best params given here

In [ ]:
for i in tqdm(train_list):
     with mlflow.start_run(run_name=f"NB WE with {i[2]}"):
        
        #Get the data in the right format
        train_x, train_y, devt_x, devt_y, dif_temp_x, dif_temp_y = ready_NB(i[0], i[1],dev_x, dev_y, dif_x, dif_y)
        MNB.fit(train_x, train_y) #change the numbers

        #dataframes
        dev_pre = model.predict(devt_x)
        dev_pre = np.transpose(dev_pre, axes=None)
        devt_y = np.transpose(devt_y, axes=None)
        df = pd.DataFrame()
        df['True'] = devt_y
        df['Predicted'] = dev_pre
    
        df.to_csv(f"pics/NB/dev {i[2]}.csv")
        #################

        dif_pre = model.predict(dif_temp_x)
        dif_pre = np.transpose(dif_pre, axes=None)
        dif_temp_y = np.transpose(dif_temp_y, axes=None)
        df = pd.DataFrame()
        df['True'] = dif_temp_y
        df['Predicted'] = dif_pre
    
        df.to_csv(f"pics/NB/dif {i[2]}.csv")

# Find train size

In [ ]:
def reading_in(path):
    thing=[]
    for line in gzip.open(path):
        review_data = json.loads(line)
        subthing = dict()
        for key in review_data:
            subthing[key]= review_data[key]
        thing.append(subthing)
    return thing

In [ ]:
train = reading_in('../data/classification/music_reviews_train.json.gz')
dev = reading_in('../data/classification/music_reviews_dev.json.gz')
test = reading_in('../data/classification/music_reviews_test_masked.json.gz')
difficult = reading_in('../data/difficult cases/phase2_testData-masked.json.gz')

In [ ]:
train_x = [i['reviewText'] if "reviewText" in i.keys() else "" for i in train]
train_y =  [i['sentiment'] for i in train]

dev_x = [i['reviewText'] if "reviewText" in i.keys() else "" for i in dev]
dev_y = [i['sentiment'] for i in dev]

test_x = [i['reviewText'] if "reviewText" in i.keys() else "" for i in test]
test_y = [i['sentiment'] for i in test]

diff_test_x = [i['reviewText'] if "reviewText" in i.keys() else "" for i in test]
diff_test_y = [i['sentiment'] for i in test] #empty

In [ ]:
pipeline = Pipeline([
    ('CountVectorizer',CountVectorizer(ngram_range=(1, 2))),
    ('Tfidf',TfidfTransformer()),
    ('clf', MultinomialNB(alpha=1, fit_prior=False))
])#best params given here

In [ ]:
def find_train_size():
    """
    90% train, 10% dev (test)
    """
    macro_f1 = []
    train_size = 1000
    
    while train_size <= len(train_x):
        
        temp_train_x = train_x[0:train_size]
        temp_train_y = train_y[0:train_size]

        pipeline.fit(temp_train_x, temp_train_y)
        pred_dev = pipeline.predict(dev_x)
                
        macro_f1.append((train_size, f1_score(pred_dev, dev_y, average='macro')))

        print(int(train_size))
        train_size += 1000
        
    return macro_f1

In [ ]:
macro_f1_dev= find_train_size()

In [ ]:
df = pd.DataFrame(macro_f1_dev, columns =['train_size', 'F1-macro (dev)'])
df.to_csv('macro_f1_dev.csv')

# Heat Maps

## CNN

def pre(df):
    end_df = []
    for i in df:
        pre_pro = []
        pre = i['Predicted'].tolist()
        for p in pre:
            if p > 0.5:
                pre_pro.append(1)
            else:
                pre_pro.append(0)
        i['Predicted'] = pre_pro
        end_df.append(i)
    return end_df

In [ ]:
dev = [ 'dev WordNet 500.csv', 'dev Word2Vec 500.csv','dev WebScrape 500.csv', 'dev original data 500.csv',\
    'dev WordNet 1250.csv', 'dev Word2Vec 1250.csv',  'dev WebScrape 1250.csv',  'dev original data 1250.csv',\
 'dev WordNet 2500.csv',  'dev Word2Vec 2500.csv', 'dev WebScrape 2500.csv',  'dev original data 2500.csv', \
 'dev WordNet 5000.csv',  'dev Word2Vec 5000.csv',  'dev WebScrape 5000.csv',  'dev original data 5000.csv',\
 'dev WordNet 7500.csv',  'dev Word2Vec 7500.csv', 'dev WebScrape 7500.csv', 'dev original data 7500.csv',\
 'dev WordNet 10000.csv', 'dev Word2Vec 10000.csv', 'dev WebScrape 10000.csv', 'dev original data 10000.csv'\
]

dif =  [ 'dif WordNet 500.csv', 'dif Word2Vec 500.csv','dif WebScrape 500.csv', 'dif original data 500.csv',\
 'dif WordNet 1250.csv', 'dif Word2Vec 1250.csv',  'dif WebScrape 1250.csv',  'dif original data 1250.csv',\
 'dif WordNet 2500.csv',  'dif Word2Vec 2500.csv', 'dif WebScrape 2500.csv',  'dif original data 2500.csv', \
 'dif WordNet 5000.csv',  'dif Word2Vec 5000.csv',  'dif WebScrape 5000.csv',  'dif original data 5000.csv',\
 'dif WordNet 7500.csv',  'dif Word2Vec 7500.csv', 'dif WebScrape 7500.csv', 'dif original data 7500.csv',\
 'dif WordNet 10000.csv', 'dif Word2Vec 10000.csv', 'dif WebScrape 10000.csv', 'dif original data 10000.csv'\
]

In [ ]:
dif_p = [pd.read_csv('pics/CNN'+i) for i in dif]
dev_p = [pd.read_csv('pics/CNN'+i) for i in dev]

In [ ]:
dif_p = pre(dif_p)
dev_p = pre(dev_p)

In [ ]:
fig, axes = plt.subplots(6, 4, figsize=(25, 25), sharey=True)
cmap = sns.color_palette("Blues", as_cmap=True)


sns.heatmap(confusion_matrix(dif_p[0]['True'], dif_p[0]['Predicted']), cmap = cmap ,ax = axes[0,0], annot=True, fmt="d", vmin = 0, vmax = 1000).set(title= dif[0][4:-4])
sns.heatmap(confusion_matrix(dif_p[1]['True'], dif_p[1]['Predicted']), cmap = cmap ,ax = axes[0,1], annot=True, fmt="d", vmin = 0, vmax = 1000).set(title= dif[1][4:-4])
sns.heatmap(confusion_matrix(dif_p[2]['True'], dif_p[2]['Predicted']), cmap = cmap ,ax = axes[0,2], annot=True, fmt="d", vmin = 0, vmax = 1000).set(title= dif[2][4:-4])
sns.heatmap(confusion_matrix(dif_p[3]['True'], dif_p[3]['Predicted']), cmap = cmap ,ax = axes[0,3], annot=True, fmt="d", vmin = 0, vmax = 1000).set(title= dif[3][4:-4])

sns.heatmap(confusion_matrix(dif_p[4]['True'], dif_p[4]['Predicted']), cmap = cmap ,ax = axes[1,0], annot=True, fmt="d", vmin = 0, vmax = 1000).set(title= dif[4][4:-4])
sns.heatmap(confusion_matrix(dif_p[5]['True'], dif_p[5]['Predicted']), cmap = cmap ,ax = axes[1,1], annot=True, fmt="d", vmin = 0, vmax = 1000).set(title= dif[5][4:-4])
sns.heatmap(confusion_matrix(dif_p[6]['True'], dif_p[6]['Predicted']), cmap = cmap ,ax = axes[1,2], annot=True, fmt="d", vmin = 0, vmax = 1000).set(title= dif[6][4:-4])
sns.heatmap(confusion_matrix(dif_p[7]['True'], dif_p[7]['Predicted']), cmap = cmap ,ax = axes[1,3], annot=True, fmt="d", vmin = 0, vmax = 1000).set(title= dif[7][4:-4])

sns.heatmap(confusion_matrix(dif_p[8]['True'], dif_p[8]['Predicted']), cmap = cmap ,ax = axes[2,0], annot=True, fmt="d", vmin = 0, vmax = 1000).set(title= dif[8][4:-4])
sns.heatmap(confusion_matrix(dif_p[9]['True'], dif_p[9]['Predicted']), cmap = cmap ,ax = axes[2,1], annot=True, fmt="d", vmin = 0, vmax = 1000).set(title= dif[9][4:-4])
sns.heatmap(confusion_matrix(dif_p[10]['True'], dif_p[10]['Predicted']),cmap = cmap , ax = axes[2,2], annot=True, fmt="d", vmin = 0, vmax = 1000).set(title= dif[10][4:-4])
sns.heatmap(confusion_matrix(dif_p[11]['True'], dif_p[11]['Predicted']),cmap = cmap , ax = axes[2,3], annot=True, fmt="d", vmin = 0, vmax = 1000).set(title= dif[11][4:-4])

sns.heatmap(confusion_matrix(dif_p[12]['True'], dif_p[12]['Predicted']),cmap = cmap , ax = axes[3,0], annot=True, fmt="d", vmin = 0, vmax = 1000).set(title= dif[12][4:-4])
sns.heatmap(confusion_matrix(dif_p[13]['True'], dif_p[13]['Predicted']),cmap = cmap , ax = axes[3,1], annot=True, fmt="d", vmin = 0, vmax = 1000).set(title= dif[13][4:-4])
sns.heatmap(confusion_matrix(dif_p[14]['True'], dif_p[14]['Predicted']),cmap = cmap , ax = axes[3,2], annot=True, fmt="d", vmin = 0, vmax = 1000).set(title= dif[14][4:-4])
sns.heatmap(confusion_matrix(dif_p[15]['True'], dif_p[15]['Predicted']),cmap = cmap , ax = axes[3,3], annot=True, fmt="d", vmin = 0, vmax = 1000).set(title= dif[15][4:-4])

sns.heatmap(confusion_matrix(dif_p[16]['True'], dif_p[16]['Predicted']), cmap = cmap ,ax = axes[4,0], annot=True, fmt="d", vmin = 0, vmax = 1000).set(title= dif[16][4:-4])
sns.heatmap(confusion_matrix(dif_p[17]['True'], dif_p[17]['Predicted']), cmap = cmap ,ax = axes[4,1], annot=True, fmt="d", vmin = 0, vmax = 1000).set(title= dif[17][4:-4])
sns.heatmap(confusion_matrix(dif_p[18]['True'], dif_p[18]['Predicted']), cmap = cmap ,ax = axes[4,2], annot=True, fmt="d", vmin = 0, vmax = 1000).set(title= dif[18][4:-4])
sns.heatmap(confusion_matrix(dif_p[19]['True'], dif_p[19]['Predicted']), cmap = cmap ,ax = axes[4,3], annot=True, fmt="d", vmin = 0, vmax = 1000).set(title= dif[19][4:-4])

sns.heatmap(confusion_matrix(dif_p[20]['True'], dif_p[20]['Predicted']),cmap = cmap , ax = axes[5,0], annot=True, fmt="d", vmin = 0, vmax = 1000).set(title= dif[20][4:-4])
sns.heatmap(confusion_matrix(dif_p[21]['True'], dif_p[21]['Predicted']),cmap = cmap , ax = axes[5,1], annot=True, fmt="d", vmin = 0, vmax = 1000).set(title= dif[21][4:-4])
sns.heatmap(confusion_matrix(dif_p[22]['True'], dif_p[22]['Predicted']),cmap = cmap , ax = axes[5,2], annot=True, fmt="d", vmin = 0, vmax = 1000).set(title= dif[22][4:-4])
sns.heatmap(confusion_matrix(dif_p[23]['True'], dif_p[23]['Predicted']),cmap = cmap , ax = axes[5,3], annot=True, fmt="d", vmin = 0, vmax = 1000).set(title= dif[23][4:-4])


plt.savefig('pics/maps/CNN/Difficult cases CNN')

In [ ]:
fig, axes = plt.subplots(6, 4, figsize=(25, 25), sharey=True)

sns.heatmap(confusion_matrix(dev_p[0]['True'], dev_p[0]['Predicted']), cmap = cmap ,ax = axes[0,0], annot=True, fmt="d", vmin = 0, vmax = 5000).set(title= dev[0][4:-4])
sns.heatmap(confusion_matrix(dev_p[1]['True'], dev_p[1]['Predicted']), cmap = cmap ,ax = axes[0,1], annot=True, fmt="d", vmin = 0, vmax = 5000).set(title= dev[1][4:-4])
sns.heatmap(confusion_matrix(dev_p[2]['True'], dev_p[2]['Predicted']), cmap = cmap ,ax = axes[0,2], annot=True, fmt="d", vmin = 0, vmax = 5000).set(title= dev[2][4:-4])
sns.heatmap(confusion_matrix(dev_p[3]['True'], dev_p[3]['Predicted']), cmap = cmap ,ax = axes[0,3], annot=True, fmt="d", vmin = 0, vmax = 5000).set(title= dev[3][4:-4])

sns.heatmap(confusion_matrix(dev_p[4]['True'], dev_p[4]['Predicted']), cmap = cmap ,ax = axes[1,0], annot=True, fmt="d", vmin = 0, vmax = 5000).set(title= dev[4][4:-4])
sns.heatmap(confusion_matrix(dev_p[5]['True'], dev_p[5]['Predicted']), cmap = cmap ,ax = axes[1,1], annot=True, fmt="d", vmin = 0, vmax = 5000).set(title= dev[5][4:-4])
sns.heatmap(confusion_matrix(dev_p[6]['True'], dev_p[6]['Predicted']), cmap = cmap ,ax = axes[1,2], annot=True, fmt="d", vmin = 0, vmax = 5000).set(title= dev[6][4:-4])
sns.heatmap(confusion_matrix(dev_p[7]['True'], dev_p[7]['Predicted']), cmap = cmap ,ax = axes[1,3], annot=True, fmt="d", vmin = 0, vmax = 5000).set(title= dev[7][4:-4])

sns.heatmap(confusion_matrix(dev_p[8]['True'], dev_p[8]['Predicted']), cmap = cmap ,ax = axes[2,0], annot=True, fmt="d", vmin = 0, vmax = 5000).set(title= dev[8][4:-4])
sns.heatmap(confusion_matrix(dev_p[9]['True'], dev_p[9]['Predicted']), cmap = cmap ,ax = axes[2,1], annot=True, fmt="d", vmin = 0, vmax = 5000).set(title= dev[9][4:-4])
sns.heatmap(confusion_matrix(dev_p[10]['True'], dev_p[10]['Predicted']),cmap = cmap , ax = axes[2,2], annot=True, fmt="d", vmin = 0, vmax = 5000).set(title= dev[10][4:-4])
sns.heatmap(confusion_matrix(dev_p[11]['True'], dev_p[11]['Predicted']),cmap = cmap , ax = axes[2,3], annot=True, fmt="d", vmin = 0, vmax = 5000).set(title= dev[11][4:-4])

sns.heatmap(confusion_matrix(dev_p[12]['True'], dev_p[12]['Predicted']),cmap = cmap , ax = axes[3,0], annot=True, fmt="d", vmin = 0, vmax = 5000).set(title= dev[12][4:-4])
sns.heatmap(confusion_matrix(dev_p[13]['True'], dev_p[13]['Predicted']),cmap = cmap , ax = axes[3,1], annot=True, fmt="d", vmin = 0, vmax = 5000).set(title= dev[13][4:-4])
sns.heatmap(confusion_matrix(dev_p[14]['True'], dev_p[14]['Predicted']),cmap = cmap , ax = axes[3,2], annot=True, fmt="d", vmin = 0, vmax = 5000).set(title= dev[14][4:-4])
sns.heatmap(confusion_matrix(dev_p[15]['True'], dev_p[15]['Predicted']),cmap = cmap , ax = axes[3,3], annot=True, fmt="d", vmin = 0, vmax = 5000).set(title= dev[15][4:-4])

sns.heatmap(confusion_matrix(dev_p[16]['True'], dev_p[16]['Predicted']), cmap = cmap ,ax = axes[4,0], annot=True, fmt="d", vmin = 0, vmax = 5000).set(title= dev[16][4:-4])
sns.heatmap(confusion_matrix(dev_p[17]['True'], dev_p[17]['Predicted']), cmap = cmap ,ax = axes[4,1], annot=True, fmt="d", vmin = 0, vmax = 5000).set(title= dev[17][4:-4])
sns.heatmap(confusion_matrix(dev_p[18]['True'], dev_p[18]['Predicted']), cmap = cmap ,ax = axes[4,2], annot=True, fmt="d", vmin = 0, vmax = 5000).set(title= dev[18][4:-4])
sns.heatmap(confusion_matrix(dev_p[19]['True'], dev_p[19]['Predicted']), cmap = cmap ,ax = axes[4,3], annot=True, fmt="d", vmin = 0, vmax = 5000).set(title= dev[19][4:-4])

sns.heatmap(confusion_matrix(dev_p[20]['True'], dev_p[20]['Predicted']),cmap = cmap , ax = axes[5,0], annot=True, fmt="d", vmin = 0, vmax = 5000).set(title= dev[20][4:-4])
sns.heatmap(confusion_matrix(dev_p[21]['True'], dev_p[21]['Predicted']),cmap = cmap , ax = axes[5,1], annot=True, fmt="d", vmin = 0, vmax = 5000).set(title= dev[21][4:-4])
sns.heatmap(confusion_matrix(dev_p[22]['True'], dev_p[22]['Predicted']),cmap = cmap , ax = axes[5,2], annot=True, fmt="d", vmin = 0, vmax = 5000).set(title= dev[22][4:-4])
sns.heatmap(confusion_matrix(dev_p[23]['True'], dev_p[23]['Predicted']),cmap = cmap , ax = axes[5,3], annot=True, fmt="d", vmin = 0, vmax = 5000).set(title= dev[23][4:-4])


plt.savefig('pics/maps/CNN/Validation cases CNN')

## Naive Bayes

def pre(df):
    end_df = []

    for i in df:
        print(i.columns)
        pre_pro = []

        pre = i['Predicted'].tolist()
        for p in pre:
            if p == 'positive':
                pre_pro.append(1)
            else:
                pre_pro.append(0)
        
        true_pro = []
        true = i['True'].tolist()
        for p in true:
            if p == 'positive':
                true_pro.append(1)
            else:
                true_pro.append(0)

        i['Predicted'] = pre_pro
        i['True'] = true_pro
        end_df.append(i)
    return end_df

In [ ]:
dev = [ 'dev WordNet 500.csv', 'dev Word2Vec 500.csv','dev WebScrape 500.csv', 'dev original data 500.csv',\
    'dev WordNet 1250.csv', 'dev Word2Vec 1250.csv',  'dev WebScrape 1250.csv',  'dev original data 1250.csv',\
 'dev WordNet 2500.csv',  'dev Word2Vec 2500.csv', 'dev WebScrape 2500.csv',  'dev original data 2500.csv', \
 'dev WordNet 5000.csv',  'dev Word2Vec 5000.csv',  'dev WebScrape 5000.csv',  'dev original data 5000.csv',\
 'dev WordNet 7500.csv',  'dev Word2Vec 7500.csv', 'dev WebScrape 7500.csv', 'dev original data 7500.csv',\
 'dev WordNet 10000.csv', 'dev Word2Vec 10000.csv', 'dev WebScrape 10000.csv', 'dev original data 10000.csv'\
]

dif =  [ 'dif WordNet 500.csv', 'dif Word2Vec 500.csv','dif WebScrape 500.csv', 'dif original data 500.csv',\
 'dif WordNet 1250.csv', 'dif Word2Vec 1250.csv',  'dif WebScrape 1250.csv',  'dif original data 1250.csv',\
 'dif WordNet 2500.csv',  'dif Word2Vec 2500.csv', 'dif WebScrape 2500.csv',  'dif original data 2500.csv', \
 'dif WordNet 5000.csv',  'dif Word2Vec 5000.csv',  'dif WebScrape 5000.csv',  'dif original data 5000.csv',\
 'dif WordNet 7500.csv',  'dif Word2Vec 7500.csv', 'dif WebScrape 7500.csv', 'dif original data 7500.csv',\
 'dif WordNet 10000.csv', 'dif Word2Vec 10000.csv', 'dif WebScrape 10000.csv', 'dif original data 10000.csv'\
]

In [ ]:
dif_p = [pd.read_csv('pics/NB/'+i) for i in dif]
dev_p = [pd.read_csv('pics/NB/'+i) for i in dev]

In [ ]:
dif_p = pre(dif_p)
dev_p = pre(dev_p)

In [ ]:
fig, axes = plt.subplots(6, 4, figsize=(25, 25), sharey=True)
fig.suptitle('Difficult cases')
cmap = sns.color_palette("Blues", as_cmap=True)


sns.heatmap(confusion_matrix(dif_p[0]['True'], dif_p[0]['Predicted']), cmap = cmap ,ax = axes[0,0], annot=True, fmt="d", vmin = 0, vmax = 1000).set(title= dif[0][4:-4])
sns.heatmap(confusion_matrix(dif_p[1]['True'], dif_p[1]['Predicted']), cmap = cmap ,ax = axes[0,1], annot=True, fmt="d", vmin = 0, vmax = 1000).set(title= dif[1][4:-4])
sns.heatmap(confusion_matrix(dif_p[2]['True'], dif_p[2]['Predicted']), cmap = cmap ,ax = axes[0,2], annot=True, fmt="d", vmin = 0, vmax = 1000).set(title= dif[2][4:-4])
sns.heatmap(confusion_matrix(dif_p[3]['True'], dif_p[3]['Predicted']), cmap = cmap ,ax = axes[0,3], annot=True, fmt="d", vmin = 0, vmax = 1000).set(title= dif[3][4:-4])

sns.heatmap(confusion_matrix(dif_p[4]['True'], dif_p[4]['Predicted']), cmap = cmap ,ax = axes[1,0], annot=True, fmt="d", vmin = 0, vmax = 1000).set(title= dif[4][4:-4])
sns.heatmap(confusion_matrix(dif_p[5]['True'], dif_p[5]['Predicted']), cmap = cmap ,ax = axes[1,1], annot=True, fmt="d", vmin = 0, vmax = 1000).set(title= dif[5][4:-4])
sns.heatmap(confusion_matrix(dif_p[6]['True'], dif_p[6]['Predicted']), cmap = cmap ,ax = axes[1,2], annot=True, fmt="d", vmin = 0, vmax = 1000).set(title= dif[6][4:-4])
sns.heatmap(confusion_matrix(dif_p[7]['True'], dif_p[7]['Predicted']), cmap = cmap ,ax = axes[1,3], annot=True, fmt="d", vmin = 0, vmax = 1000).set(title= dif[7][4:-4])

sns.heatmap(confusion_matrix(dif_p[8]['True'], dif_p[8]['Predicted']), cmap = cmap ,ax = axes[2,0], annot=True, fmt="d", vmin = 0, vmax = 1000).set(title= dif[8][4:-4])
sns.heatmap(confusion_matrix(dif_p[9]['True'], dif_p[9]['Predicted']), cmap = cmap ,ax = axes[2,1], annot=True, fmt="d", vmin = 0, vmax = 1000).set(title= dif[9][4:-4])
sns.heatmap(confusion_matrix(dif_p[10]['True'], dif_p[10]['Predicted']),cmap = cmap , ax = axes[2,2], annot=True, fmt="d", vmin = 0, vmax = 1000).set(title= dif[10][4:-4])
sns.heatmap(confusion_matrix(dif_p[11]['True'], dif_p[11]['Predicted']),cmap = cmap , ax = axes[2,3], annot=True, fmt="d", vmin = 0, vmax = 1000).set(title= dif[11][4:-4])

sns.heatmap(confusion_matrix(dif_p[12]['True'], dif_p[12]['Predicted']),cmap = cmap , ax = axes[3,0], annot=True, fmt="d", vmin = 0, vmax = 1000).set(title= dif[12][4:-4])
sns.heatmap(confusion_matrix(dif_p[13]['True'], dif_p[13]['Predicted']),cmap = cmap , ax = axes[3,1], annot=True, fmt="d", vmin = 0, vmax = 1000).set(title= dif[13][4:-4])
sns.heatmap(confusion_matrix(dif_p[14]['True'], dif_p[14]['Predicted']),cmap = cmap , ax = axes[3,2], annot=True, fmt="d", vmin = 0, vmax = 1000).set(title= dif[14][4:-4])
sns.heatmap(confusion_matrix(dif_p[15]['True'], dif_p[15]['Predicted']),cmap = cmap , ax = axes[3,3], annot=True, fmt="d", vmin = 0, vmax = 1000).set(title= dif[15][4:-4])

sns.heatmap(confusion_matrix(dif_p[16]['True'], dif_p[16]['Predicted']), cmap = cmap ,ax = axes[4,0], annot=True, fmt="d", vmin = 0, vmax = 1000).set(title= dif[16][4:-4])
sns.heatmap(confusion_matrix(dif_p[17]['True'], dif_p[17]['Predicted']), cmap = cmap ,ax = axes[4,1], annot=True, fmt="d", vmin = 0, vmax = 1000).set(title= dif[17][4:-4])
sns.heatmap(confusion_matrix(dif_p[18]['True'], dif_p[18]['Predicted']), cmap = cmap ,ax = axes[4,2], annot=True, fmt="d", vmin = 0, vmax = 1000).set(title= dif[18][4:-4])
sns.heatmap(confusion_matrix(dif_p[19]['True'], dif_p[19]['Predicted']), cmap = cmap ,ax = axes[4,3], annot=True, fmt="d", vmin = 0, vmax = 1000).set(title= dif[19][4:-4])

sns.heatmap(confusion_matrix(dif_p[20]['True'], dif_p[20]['Predicted']),cmap = cmap , ax = axes[5,0], annot=True, fmt="d", vmin = 0, vmax = 1000).set(title= dif[20][4:-4])
sns.heatmap(confusion_matrix(dif_p[21]['True'], dif_p[21]['Predicted']),cmap = cmap , ax = axes[5,1], annot=True, fmt="d", vmin = 0, vmax = 1000).set(title= dif[21][4:-4])
sns.heatmap(confusion_matrix(dif_p[22]['True'], dif_p[22]['Predicted']),cmap = cmap , ax = axes[5,2], annot=True, fmt="d", vmin = 0, vmax = 1000).set(title= dif[22][4:-4])
sns.heatmap(confusion_matrix(dif_p[23]['True'], dif_p[23]['Predicted']),cmap = cmap , ax = axes[5,3], annot=True, fmt="d", vmin = 0, vmax = 1000).set(title= dif[23][4:-4])


plt.savefig('pics/maps/NB/Difficult cases NB')

In [ ]:
fig, axes = plt.subplots(6, 4, figsize=(25, 25), sharey=True)
fig.suptitle('Validation set')

sns.heatmap(confusion_matrix(dev_p[0]['True'], dev_p[0]['Predicted']), cmap = cmap ,ax = axes[0,0], annot=True, fmt="d", vmin = 0, vmax = 5000).set(title= dev[0][4:-4])
sns.heatmap(confusion_matrix(dev_p[1]['True'], dev_p[1]['Predicted']), cmap = cmap ,ax = axes[0,1], annot=True, fmt="d", vmin = 0, vmax = 5000).set(title= dev[1][4:-4])
sns.heatmap(confusion_matrix(dev_p[2]['True'], dev_p[2]['Predicted']), cmap = cmap ,ax = axes[0,2], annot=True, fmt="d", vmin = 0, vmax = 5000).set(title= dev[2][4:-4])
sns.heatmap(confusion_matrix(dev_p[3]['True'], dev_p[3]['Predicted']), cmap = cmap ,ax = axes[0,3], annot=True, fmt="d", vmin = 0, vmax = 5000).set(title= dev[3][4:-4])

sns.heatmap(confusion_matrix(dev_p[4]['True'], dev_p[4]['Predicted']), cmap = cmap ,ax = axes[1,0], annot=True, fmt="d", vmin = 0, vmax = 5000).set(title= dev[4][4:-4])
sns.heatmap(confusion_matrix(dev_p[5]['True'], dev_p[5]['Predicted']), cmap = cmap ,ax = axes[1,1], annot=True, fmt="d", vmin = 0, vmax = 5000).set(title= dev[5][4:-4])
sns.heatmap(confusion_matrix(dev_p[6]['True'], dev_p[6]['Predicted']), cmap = cmap ,ax = axes[1,2], annot=True, fmt="d", vmin = 0, vmax = 5000).set(title= dev[6][4:-4])
sns.heatmap(confusion_matrix(dev_p[7]['True'], dev_p[7]['Predicted']), cmap = cmap ,ax = axes[1,3], annot=True, fmt="d", vmin = 0, vmax = 5000).set(title= dev[7][4:-4])

sns.heatmap(confusion_matrix(dev_p[8]['True'], dev_p[8]['Predicted']), cmap = cmap ,ax = axes[2,0], annot=True, fmt="d", vmin = 0, vmax = 5000).set(title= dev[8][4:-4])
sns.heatmap(confusion_matrix(dev_p[9]['True'], dev_p[9]['Predicted']), cmap = cmap ,ax = axes[2,1], annot=True, fmt="d", vmin = 0, vmax = 5000).set(title= dev[9][4:-4])
sns.heatmap(confusion_matrix(dev_p[10]['True'], dev_p[10]['Predicted']),cmap = cmap , ax = axes[2,2], annot=True, fmt="d", vmin = 0, vmax = 5000).set(title= dev[10][4:-4])
sns.heatmap(confusion_matrix(dev_p[11]['True'], dev_p[11]['Predicted']),cmap = cmap , ax = axes[2,3], annot=True, fmt="d", vmin = 0, vmax = 5000).set(title= dev[11][4:-4])

sns.heatmap(confusion_matrix(dev_p[12]['True'], dev_p[12]['Predicted']),cmap = cmap , ax = axes[3,0], annot=True, fmt="d", vmin = 0, vmax = 5000).set(title= dev[12][4:-4])
sns.heatmap(confusion_matrix(dev_p[13]['True'], dev_p[13]['Predicted']),cmap = cmap , ax = axes[3,1], annot=True, fmt="d", vmin = 0, vmax = 5000).set(title= dev[13][4:-4])
sns.heatmap(confusion_matrix(dev_p[14]['True'], dev_p[14]['Predicted']),cmap = cmap , ax = axes[3,2], annot=True, fmt="d", vmin = 0, vmax = 5000).set(title= dev[14][4:-4])
sns.heatmap(confusion_matrix(dev_p[15]['True'], dev_p[15]['Predicted']),cmap = cmap , ax = axes[3,3], annot=True, fmt="d", vmin = 0, vmax = 5000).set(title= dev[15][4:-4])

sns.heatmap(confusion_matrix(dev_p[16]['True'], dev_p[16]['Predicted']), cmap = cmap ,ax = axes[4,0], annot=True, fmt="d", vmin = 0, vmax = 5000).set(title= dev[16][4:-4])
sns.heatmap(confusion_matrix(dev_p[17]['True'], dev_p[17]['Predicted']), cmap = cmap ,ax = axes[4,1], annot=True, fmt="d", vmin = 0, vmax = 5000).set(title= dev[17][4:-4])
sns.heatmap(confusion_matrix(dev_p[18]['True'], dev_p[18]['Predicted']), cmap = cmap ,ax = axes[4,2], annot=True, fmt="d", vmin = 0, vmax = 5000).set(title= dev[18][4:-4])
sns.heatmap(confusion_matrix(dev_p[19]['True'], dev_p[19]['Predicted']), cmap = cmap ,ax = axes[4,3], annot=True, fmt="d", vmin = 0, vmax = 5000).set(title= dev[19][4:-4])

sns.heatmap(confusion_matrix(dev_p[20]['True'], dev_p[20]['Predicted']),cmap = cmap , ax = axes[5,0], annot=True, fmt="d", vmin = 0, vmax = 5000).set(title= dev[20][4:-4])
sns.heatmap(confusion_matrix(dev_p[21]['True'], dev_p[21]['Predicted']),cmap = cmap , ax = axes[5,1], annot=True, fmt="d", vmin = 0, vmax = 5000).set(title= dev[21][4:-4])
sns.heatmap(confusion_matrix(dev_p[22]['True'], dev_p[22]['Predicted']),cmap = cmap , ax = axes[5,2], annot=True, fmt="d", vmin = 0, vmax = 5000).set(title= dev[22][4:-4])
sns.heatmap(confusion_matrix(dev_p[23]['True'], dev_p[23]['Predicted']),cmap = cmap , ax = axes[5,3], annot=True, fmt="d", vmin = 0, vmax = 5000).set(title= dev[23][4:-4])


plt.savefig('pics/maps/NB/Validation cases')